# Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import datetime
import time
import dateutil.parser
import requests
from typing import List, Dict

%matplotlib inline
plt.rcParams['figure.figsize'] = [17, 7]

# Get Order Book Data

In [9]:
def orderbook_data(depth: int, contract: str = "ETHUSD", raw_data=False):

    url = "https://www.bitmex.com/api/v1/orderBook/L2"
    book_data = {'buys': 0, 'sells': 0}
    raw_book_data = {'total_buys': 0, 'buys': 0, 'total_sells': 0, 'sells': 0}

    # Set inputs
    data = dict()
    data['symbol'] = contract
    data['depth'] = depth

    try:
        response = requests.get(url, params=data)
    except Exception as e:
        print(f"Connecton error while making GET request to {url}: {e}")
        return

    if response.status_code == 200:
        data = response.json()
    else:
        print(f"Error while making GET request to {url}: {response.status_code}")
        print(response.headers)
        return None
    
    if data is not None:
        if raw_data:
            for data in data:
                if data['side'] == "Buy":
                    raw_book_data["buys"] += data["size"]
                    raw_book_data['total_buys'] += 1

                elif data['side'] == "Sell":
                    raw_book_data["sells"] += data["size"]
                    raw_book_data['total_sells'] += 1

            return raw_book_data

        else:
            for data in data:
                if data['side'] == "Buy":
                    book_data["buys"] += data["size"]

                elif data['side'] == "Sell":
                    book_data["sells"] += data["size"]
    
            return book_data

In [11]:
depth = 500
contract = "XBTUSD"

book_data = orderbook_data(depth, contract, raw_data=False)
book_size = orderbook_data(0, contract, raw_data=True)
book_size = (len(book_size["total_buys"]) + len(book_size["total_sells"])) // 2

ret = "Buyers" if book_data["buys"] > book_data["sells"] else "sellers"

if ret == "Buyers":
    amount = book_data["buys"] - book_data["sells"]
else:
    amount = book_data["sells"] - book_data["buys"]


print(book_data)
print(f"\nThere are {amount} more order values from {ret}")
print(f"\nThe size of the order book for {contract} is {book_size}")

TypeError: list indices must be integers or slices, not str